In [1]:
from src.lobbyingDataPage import *
from src.lobbyingScraper import *

import pickle
with open("tests/testfiles/2005html.pkl", "rb") as f:
    html05 = pickle.load(f)
with open("tests/testfiles/2020html.pkl", "rb") as f:
    html20 = pickle.load(f)
test_htmls = html05.__add__(html20)

In [2]:
#%%capture
pages = convert_html_list(test_htmls)

converting html index 0 into data page
converting html index 1 into data page
converting html index 2 into data page
converting html index 3 into data page
converting html index 4 into data page
converting html index 5 into data page
converting html index 6 into data page
converting html index 7 into data page
converting html index 8 into data page
converting html index 9 into data page
converting html index 10 into data page
converting html index 11 into data page
converting html index 12 into data page
converting html index 13 into data page
converting html index 14 into data page
converting html index 15 into data page
converting html index 16 into data page
converting html index 17 into data page
converting html index 18 into data page
converting html index 19 into data page
converting html index 20 into data page
converting html index 21 into data page
converting html index 22 into data page
converting html index 23 into data page
converting html index 24 into data page
converting

KeyboardInterrupt: 

In [2]:
html = test_htmls[69]
EntityDataPage(html)

['12/1/05Timothy A Bassett', 'Tom McGee', 'Senator of Lynn', '$50.00']


In [11]:
soup = bs(html, 'html.parser')

In [12]:
table_name = 'Campaign Contributions'
columns = ['Date','Lobbyist name','Recipient name','Office sought','Amount']
table_start = "".join(columns)
table_end = "\xa0\xa0Total contributions"

In [16]:
# pulls everything between each tabe_start and table_end
# returns a list of results
def query_page(table_start, table_end, string):
    query_string = fr"(?<={table_start}).*?(?={table_end})"
    query = re.compile(query_string,re.DOTALL)
    query_results = re.findall(query, string)
    return query_results
def divide_chunks(some_list,chunk_size):
    for i in range(0, len(some_list), chunk_size):
        yield some_list[i:i+chunk_size]

In [24]:
query_results = query_page(table_start, table_end, soup.text)
query_result = query_results[0]
split_text=[line.strip() for line in query_result.split('\n') if line.strip()]
divided_text = list(divide_chunks(split_text, len(columns)))
if len(divided_text) < len(columns):
    print(divided_text[0])
len(divided_text[0])
# def create_table(query_result, columns):
#     split_text=[line.strip() for line in query_result.split('\n') if line.strip()]
#     divided_text = list(divide_chunks(split_text, len(columns)))
#     table_df = pd.DataFrame(divided_text, columns=columns)
#     return table_df

['12/1/05Timothy A Bassett', 'Tom McGee', 'Senator of Lynn', '$50.00']


4

In [6]:
comp = [page for page in pages if 'Client Compensation' in page.soup.text]
comp

In [13]:
comp[0].soup.text

"\n\n\n\n\n\n\n\n\r\n\tLobbyist Public Search\r\n\n\n\n\n\n  \n\n\n\n\nHome\nDirections\nContact Us\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLobbyist Public Search\n\n\n\n\n\n\n\n\n\r\n            Records prior to registration year 2005 are not available online. \r\n            \n\r\n            \r\n            Contact the Lobbyist Division here for assistance.\r\n        \nSearch data is collected from registration records and disclosure reports \r\n        submitted by lobbyists, lobbying entities, and their clients. Massachusetts General Law requires any party providing or retaining \r\n        lobbying services to register annually, and submit disclosure reports twice a year.  \r\n        \n\n\nYour comments are welcome:\xa0\r\n            submit feedback\n\n\n\n\n\n\n\n\n\n\n\nDisclosure reporting details\n\n\n\nBack to search detail summary\n\n\n\n\nLobbyist Entity disclosure reporting period:\r\n                        \xa0 \r\n                        07/01/2020 - 12/31/2020

In [7]:
set([page.tables['Headers']['Agent type'][1] for page in pages])

{'Both', 'ExecutiveAgent', 'LegislativeAgent', 'None', nan}

In [3]:
possible_tables = ['Headers','Campaign Contributions','Activities','Client Compensation']
[page.tables.keys() for page in pages if possible_tables[3] in page.tables.keys()]

[]

In [3]:
columns =['Authorizing Officer name','Lobbyist name','Title','Business name','Address','City, state, zip code','Country','Agent type','Phone']
pd.DataFrame(columns=columns)

,Authorizing Officer name,Lobbyist name,Title,Business name,Address,"City, state, zip code",Country,Agent type,Phone


In [54]:
page = pages[0]
columns =['Authorizing Officer name','Lobbyist name','Title','Business name','Address','City, state, zip code','Country','Agent type','Phone']
header_df = page.dfs[5][0:7].transpose() #Extract header table and orient it properly
header_df.columns = header_df.iloc[0] #Pull the column names from the first row...
header_df = header_df[1:] # ... and then drop that row
page.update_table('Headers', header_df)


In [55]:
page.soup.text

'\n\n\n\n\n\n\n\n\r\n\tLobbyist Public Search\r\n\n\n\n\n\n  \n\n\n\n\nHome\nDirections\nContact Us\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLobbyist Public Search\n\n\n\n\n\n\n\n\n\r\n            Records prior to registration year 2005 are not available online. \r\n            \n\r\n            \r\n            Contact the Lobbyist Division here for assistance.\r\n        \nSearch data is collected from registration records and disclosure reports \r\n        submitted by lobbyists, lobbying entities, and their clients. Massachusetts General Law requires any party providing or retaining \r\n        lobbying services to register annually, and submit disclosure reports twice a year.  \r\n        \n\n\nYour comments are welcome:\xa0\r\n            submit feedback\n\n\n\n\n\n\n\n\n\n\n\nDisclosure reporting details\n\n\n\nBack to search detail summary\n\n\n\n\nLobbyist disclosure reporting period:\r\n                        \xa0 \r\n                        01/01/2005 - 06/30/2005\n\n\n\

In [9]:
from IPython.display import HTML

HTML(html)

TypeError: startswith first arg must be bytes or a tuple of bytes, not str

In [ ]:
# Header:
# Authorizing Officer name / Lobbyist name
# Title / NA
# Business name / Business name
# Address / Address
# City, state, zip code / Citym, state, zip code
# country / country
# NA / Agent Type

#TABLES:
# Entities
# Lobbyists
# Client Compensation
# Compensation/Salaries Paid
# Activities, Bill Numbers and Titles
# Operating Expenses
# Meals, Travel, and Entertainment Expenses
# Additional Expenses
# Campaign Contributions